# Loading Set

In [1]:
from keras import losses, metrics
from keras.models import Sequential
from keras.models import Model, load_model
from keras.optimizers import Adam as Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPool1D, BatchNormalization, Dropout, Activation
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D
from keras.layers import Input, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import os, sys, pickle
import pandas as pd
import random

In [2]:
import pandas as pd

print('loading train...', flush=True, end='')

# x를 loading해서 (batch_size, step, channel)
input_path = '../dataset/preprocess4/input_120min/'
x_train = np.load(input_path+'x_train.npz', allow_pickle=True)['arr_0']
x_test = np.load(input_path+'x_test.npz', allow_pickle=True)['arr_0']
x_val = np.load(input_path+'x_val.npz', allow_pickle=True)['arr_0']

y_train = np.load(input_path+'y_train.npz')['arr_0']
y_test = np.load(input_path+'y_test.npz')['arr_0']
y_val = np.load(input_path+'y_val.npz')['arr_0']


input_path = '../dataset/preprocess4/input_120min/'
gender_train = np.load(input_path+'gender_train.npz', allow_pickle=True)['arr_0']
gender_test = np.load(input_path+'gender_test.npz', allow_pickle=True)['arr_0']
gender_val = np.load(input_path+'gender_val.npz', allow_pickle=True)['arr_0']

age_train = np.load(input_path+'age_train.npz', allow_pickle=True)['arr_0']
age_test = np.load(input_path+'age_test.npz', allow_pickle=True)['arr_0']
age_val = np.load(input_path+'age_val.npz', allow_pickle=True)['arr_0']

print('done', flush=True)

print('x_train shape:', x_train.shape)
print('x_test.shape:', x_test.shape)
print('x_val.shape:', x_val.shape)

# binary classification
y_train_bin = y_train >= 4
y_test_bin = y_test >= 4
y_val_bin = y_val >= 4

loading train...done
x_train shape: (172571, 5000, 2)
x_test.shape: (20585, 5000, 2)
x_val.shape: (18701, 5000, 2)


## pacu set for val, train set & sample weights(3 class)

In [3]:
x_test_pacu = np.load(input_path+'x_test_pacu.npz', allow_pickle=True)['arr_0']
y_test_pacu = np.load(input_path+'y_test_pacu.npz')['arr_0']
x_val_pacu = np.load(input_path+'x_val_pacu.npz', allow_pickle=True)['arr_0']
y_val_pacu =  np.load(input_path+'y_val_pacu.npz')['arr_0']

y_test_bin = y_test_pacu>=4
y_val_bin = y_val_pacu>=4


x_val_pacu = np.transpose(x_val_pacu, [0,2,1])
x_val_pacu[:,:,0] = pd.DataFrame(x_val_pacu[:,:,0]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
x_val_pacu[:,:,1] = pd.DataFrame(x_val_pacu[:,:,1]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values

x_test_pacu = np.transpose(x_test_pacu, [0,2,1])
x_test_pacu[:,:,0] = pd.DataFrame(x_test_pacu[:,:,0]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
x_test_pacu[:,:,1] = pd.DataFrame(x_test_pacu[:,:,1]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values


# train: age+gender
age_train = np.load(input_path+'age_train.npz', allow_pickle=True)['arr_0']
gender_train = np.load(input_path+'gender_train.npz', allow_pickle=True)['arr_0']

agender_train = np.array([[age_train[i], gender_train[i]] for i in range(len(age_train))])


# val: age+gender
age_val_pacu = np.load(input_path+'age_val_pacu.npz', allow_pickle=True)['arr_0']
gender_val_pacu = np.load(input_path+'gender_val_pacu.npz', allow_pickle=True)['arr_0']

agender_val_pacu = np.array([[age_val_pacu[i], gender_val_pacu[i]] for i in range(len(age_val_pacu))])


# test: age+gender
age_test_pacu = np.load(input_path+'age_test_pacu.npz', allow_pickle=True)['arr_0']
gender_test_pacu = np.load(input_path+'gender_test_pacu.npz', allow_pickle=True)['arr_0']

agender_test_pacu = np.array([[age_test_pacu[i], gender_test_pacu[i]] for i in range(len(age_test_pacu))])


# 3 class에 대한 sample weight
train_w_samp3 = np.ones(shape=(len(y_train),))

train_w_samp3[y_train<4]= len(y_train)/np.sum(y_train<4)
train_w_samp3[(4<=y_train)&(y_train<7)]= len(y_train)/np.sum((4<=y_train)&(y_train<7))
train_w_samp3[7<=y_train]= len(y_train)/np.sum(7<=y_train)

print('train set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\n'
      .format(len(y_train)/np.sum(y_train<4),len(y_train)/np.sum((4<=y_train)&(y_train<7)),len(y_train)/np.sum(7<=y_train)))


# val sample weight
val_w_samp3 = np.ones(shape=(len(y_val_pacu),))

val_w_samp3[y_val_pacu<4]= len(y_val_pacu)/np.sum(y_val_pacu<4)
val_w_samp3[(4<=y_val_pacu)&(y_val_pacu<7)]= len(y_val_pacu)/np.sum((4<=y_val_pacu)&(y_val_pacu<7))
val_w_samp3[7<=y_val_pacu]= len(y_val_pacu)/np.sum(7<=y_val_pacu)

print('val_pacu set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\n'
      .format(len(y_val_pacu)/np.sum(y_val_pacu<4),len(y_val_pacu)/np.sum((4<=y_val_pacu)&(y_val_pacu<7)),len(y_val_pacu)/np.sum(7<=y_val_pacu)))


# test sample weight
test_w_samp3 = np.ones(shape=(len(y_test_pacu),))

test_w_samp3[y_test_pacu<4]= len(y_test_pacu)/np.sum(y_test_pacu<4)
test_w_samp3[(4<=y_test_pacu)&(y_test_pacu<7)]= len(y_test_pacu)/np.sum((4<=y_test_pacu)&(y_test_pacu<7))
test_w_samp3[7<=y_test_pacu]= len(y_test_pacu)/np.sum(7<=y_test_pacu)

print('test_pacu set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}'
      .format(len(y_test_pacu)/np.sum(y_test_pacu<4),len(y_test_pacu)/np.sum((4<=y_test_pacu)&(y_test_pacu<7)),len(y_test_pacu)/np.sum(7<=y_test_pacu)))

train set
sample weight for class 1: 2.84, class 2: 2.07, class 3: 6.08

val_pacu set
sample weight for class 1: 3.90, class 2: 1.77, class 3: 5.61

test_pacu set
sample weight for class 1: 3.64, class 2: 1.85, class 3: 5.42


## Sample weight

### 2 class (NRS>=4, NRS<4)

In [ ]:
# 2 class에 대한 sample weight
train_w_samp2 = np.ones(shape=(len(y_train),))
train_w_samp2[y_train_bin==0]= len(y_train) / np.sum(y_train_bin)
train_w_samp2[y_train_bin!=0]= len(y_train) / np.sum(~y_train_bin)

train_w_samp2_2 = np.ones(shape=(len(y_train),))
train_w_samp2_2[y_train_bin==0]= len(y_train) / np.sum(y_train_bin2)
train_w_samp2_2[y_train_bin!=0]= len(y_train) / np.sum(~y_train_bin2)

print('train set')
print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_train) / np.sum(y_train_bin), len(y_train) / np.sum(~y_train_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}\n'
      .format(len(y_train) / np.sum(y_train_bin2), len(y_train) / np.sum(~y_train_bin2)))

# 2 class에 대한 sample weight
val_w_samp2 = np.ones(shape=(len(y_val),))
val_w_samp2[y_val_bin==0]= len(y_val) / np.sum(y_val_bin)
val_w_samp2[y_val_bin!=0]= len(y_val) / np.sum(~y_val_bin)

val_w_samp2_2 = np.ones(shape=(len(y_val),))
val_w_samp2_2[y_val_bin==0]= len(y_val) / np.sum(y_val_bin2)
val_w_samp2_2[y_val_bin!=0]= len(y_val) / np.sum(~y_val_bin2)

print('val set')
print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_val) / np.sum(y_val_bin), len(y_val) / np.sum(~y_val_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}\n'
      .format(len(y_val) / np.sum(y_val_bin2), len(y_val) / np.sum(~y_val_bin2)))

# 2 class에 대한 sample weight
test_w_samp2 = np.ones(shape=(len(y_test),))
test_w_samp2[y_test_bin==0]= len(y_test) / np.sum(y_test_bin)
test_w_samp2[y_test_bin!=0]= len(y_test) / np.sum(~y_test_bin)

test_w_samp2_2 = np.ones(shape=(len(y_test),))
test_w_samp2_2[y_test_bin==0]= len(y_test) / np.sum(y_test_bin2)
test_w_samp2_2[y_test_bin!=0]= len(y_test) / np.sum(~y_test_bin2)

print('test set')
print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_test) / np.sum(y_test_bin), len(y_test) / np.sum(~y_test_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}'
      .format(len(y_test) / np.sum(y_test_bin2), len(y_test) / np.sum(~y_test_bin2)))

### 3 class (NRS>=7, 7>NRS>=4, NRS<4)

In [3]:
# 3 class에 대한 sample weight
train_w_samp3 = np.ones(shape=(len(y_train),))

train_w_samp3[y_train<4]= len(y_train)/np.sum(y_train<4)
train_w_samp3[(4<=y_train)&(y_train<7)]= len(y_train)/np.sum((4<=y_train)&(y_train<7))
train_w_samp3[7<=y_train]= len(y_train)/np.sum(7<=y_train)

print('train set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\m'
      .format(len(y_train)/np.sum(y_train<4),len(y_train)/np.sum((4<=y_train)&(y_train<7)),len(y_train)/np.sum(7<=y_train)))


val_w_samp3 = np.ones(shape=(len(y_val),))

val_w_samp3[y_val<4]= len(y_val)/np.sum(y_val<4)
val_w_samp3[(4<=y_val)&(y_val<7)]= len(y_val)/np.sum((4<=y_val)&(y_val<7))
val_w_samp3[7<=y_val]= len(y_val)/np.sum(7<=y_val)

print('val set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\m'
      .format(len(y_val)/np.sum(y_val<4),len(y_val)/np.sum((4<=y_val)&(y_val<7)),len(y_val)/np.sum(7<=y_val)))


test_w_samp3 = np.ones(shape=(len(y_test),))

test_w_samp3[y_test<4]= len(y_test)/np.sum(y_test<4)
test_w_samp3[(4<=y_test)&(y_test<7)]= len(y_test)/np.sum((4<=y_test)&(y_test<7))
test_w_samp3[7<=y_test]= len(y_test)/np.sum(7<=y_test)

print('test set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}'
      .format(len(y_test)/np.sum(y_test<4),len(y_test)/np.sum((4<=y_test)&(y_test<7)),len(y_test)/np.sum(7<=y_test)))


train set
sample weight for class 1: 2.98, class 2: 2.00, class 3: 6.07\m
val set
sample weight for class 1: 2.91, class 2: 1.99, class 3: 6.53\m
test set
sample weight for class 1: 2.73, class 2: 2.10, class 3: 6.36


# Settings

In [4]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

4 Physical GPUs, 4 Logical GPUs


In [5]:
# folder
nfold = 1  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 100
rootdir = "ECG/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_120min"

predirs = []
for root, dirs, files in os.walk(rootdir):  # 하위 대상들을 recursive 하게 긁어옴
    for filename in dirs:
        predirs.append(filename)

if not os.path.exists(rootdir):
    os.mkdir(rootdir)


# test_settings
test_settings_1, test_settings_2, test_settings_3, test_settings_4 = [], [], [], []


# hyperparamters
#num_nodes = [64, 64, 64] #, 64, 64, 64]
#kernel_size = 10
pool_size = 2

#dense_node = 32
#dropout_rate = 0.2
learning_rate = 0.002

# hyperparamters pool
num_opts = [32, 64, 128, 256] # num of filters(kernel)
stride_opts = [1,1,1,1,1,2,2,2,2,3]
kernel_opts = [3,3,3,3,3,5,5,5,5,7] # kernel size
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [16, 32, 64, 128]
globalpool_opts = ['max','ave']
BATCH_SIZE = [512, 1024]


print('start making test settings...', end='', flush=True)
# test settings
for num_l1 in num_opts:
    for num_l2 in num_opts:
        for num_l3 in num_opts:
            for num_l4 in num_opts:
                for kernel_l1 in kernel_opts:
                    for kernel_l2 in kernel_opts:
                        for kernel_l3 in kernel_opts:
                            for kernel_l4 in kernel_opts:
                                test_settings_1.append([num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4])

for dense1 in dense_opts:
    for dropout1 in dropout_opts:
        for dropout_fc in dropout_opts:
            for globalpool_opt in globalpool_opts:
                for batch_size in BATCH_SIZE:
                    for conv_double in [True, False]:
                        test_settings_2.append([dense1, dropout1, dropout_fc, globalpool_opt, batch_size, conv_double])                                   

for stride_l1 in stride_opts:
    for stride_l2 in stride_opts:
        for stride_l3 in stride_opts:
            for stride_l4 in stride_opts:
                for stride_l5 in stride_opts:
                    for num_l5 in num_opts:
                        for kernel_l5 in kernel_opts:
                            test_settings_3.append([stride_l1, stride_l2, stride_l3, stride_l4, stride_l5, num_l5, kernel_l5])
      
for dense2 in dense_opts:
    for dropout2 in dropout_opts:
        test_settings_4.append([dense2, dropout2])
    
print('done')

start making test settings...done


# RandomSearch

## binary

### PPG setting

In [ ]:
x_train = x_train[:,:,0:1]
x_val = x_val[:,:,0:1]
x_test = x_test[:,:,0:1]

### ECG setting

In [6]:
x_train = x_train[:,:,1:2]
x_val = x_val[:,:,1:2]
x_test = x_test[:,:,1:2]

### Input setting

In [7]:
# input, sample weight settings
train_w_samp = train_w_samp3
val_w_samp = val_w_samp3
test_w_samp = test_w_samp3

agender_train = np.array([[age_train[i], gender_train[i]] for i in range(len(age_train))])
agender_val = np.array([[age_val[i], gender_val[i]] for i in range(len(age_val))])
agender_test = np.array([[age_test[i], gender_test[i]] for i in range(len(age_test))])

x_trains = [x_train, agender_train]
x_vals = [x_val, agender_val]
x_tests = [x_test, agender_test]

### val, train pacu set

In [6]:
# ECG
train_w_samp = train_w_samp3
val_w_samp = val_w_samp3
test_w_samp = test_w_samp3


x_train = x_train[:,:,1:2]
x_val_pacu = x_val_pacu[:,:,1:2]
x_test_pacu = x_test_pacu[:,:,1:2]


x_trains = [x_train, agender_train]
x_vals = [x_val_pacu, agender_val_pacu]
x_tests = [x_test_pacu, agender_test_pacu]

In [ ]:
from keras import metrics
# random search for hyperparameter
ntrial = 400
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_acc = []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    test_setting_3 = random.choice(test_settings_3)
    test_setting_4 = random.choice(test_settings_4)    
    
        
    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense1, dropout1, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2
    stride_l1, stride_l2, stride_l3, stride_l4, stride_l5, num_l5, kernel_l5 = test_setting_3
    dense2, dropout2 = test_setting_4
    
    
    # total conv layers of the model
    n_conv = random.choice([2,3])
    
    if n_conv==2:
        num_l3,kernel_l3,stride_l3 = 0,0,0
        num_l4,kernel_l4,stride_l4 = 0,0,0
        num_l5,kernel_l5,stride_l5 = 0,0,0
    
    if n_conv==3:
        num_l4,kernel_l4,stride_l4 = 0,0,0
        num_l5,kernel_l5,stride_l5 = 0,0,0
        
    if n_conv==4:
        num_l5,kernel_l5,stride_l5 = 0,0,0  
        
    
    n_dense = random.choice([0,1,2])
    
    if n_dense==0:
        dense1, dropout1 = 0,0
        dense2, dropout2 = 0,0
    
    if n_dense==1:
        dense2, dropout2 = 0,0
        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={},c1={},c2={},c3={},filt1={},filt2={},filt3={},str1={},str2={},str3={},conv_double={},globalpool={},dropout={},dnodes={},dropout={},dnodes={},dropout={}'.format(batch_size, num_l1, num_l2, num_l3,kernel_l1, kernel_l2, kernel_l3, stride_l1,stride_l2,stride_l3,conv_double, globalpool_opt, dropout_fc, dense1, dropout1, dense2, dropout2)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/weights.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    with strategy.scope():
        # build a model
        inp_fnn = Input(shape=(agender_train.shape[1],))
        inp_cnn = Input(shape=(x_train.shape[1], x_train.shape[2]))
    
        out_fnn = inp_fnn
        out_fnn = Activation('sigmoid') (out_fnn)
        
        out_cnn = inp_cnn
        act='relu'
        
        
        # c1 layer
        if conv_double:
            out_cnn = Conv1D(filters=num_l1, kernel_size=kernel_l1, strides=stride_l1, padding='same') (out_cnn)
        out_cnn = Conv1D(filters=num_l1, kernel_size=kernel_l1, strides=stride_l1, padding='same') (out_cnn)
        out_cnn = Activation('relu') (out_cnn)
        out_cnn = BatchNormalization() (out_cnn)
        out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)


        # c2 layer
        if conv_double:
            out_cnn = Conv1D(filters=num_l2, kernel_size=kernel_l2, strides=stride_l2, padding='same') (out_cnn)
        out_cnn = Conv1D(filters=num_l2, kernel_size=kernel_l2, strides=stride_l2, padding='same') (out_cnn)
        out_cnn = Activation('relu') (out_cnn)
        out_cnn = BatchNormalization() (out_cnn)
        out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
        
        
        # c3 layer
        if n_conv>2:
            if conv_double:
                out_cnn = Conv1D(filters=num_l3, kernel_size=kernel_l3, strides=stride_l3, padding='same') (out_cnn)
            out_cnn = Conv1D(filters=num_l3, kernel_size=kernel_l3, strides=stride_l3, padding='same') (out_cnn)
            out_cnn = Activation('relu') (out_cnn)
            out_cnn = BatchNormalization() (out_cnn)
            out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
        
        
        # c4 layer
        if n_conv>3:
            if num_l3 == 512:
                model.add(Conv1D(filters=128,kernel_size=1,padding='same'))
            if conv_double:
                out_cnn = Conv1D(filters=num_l4, kernel_size=kernel_l4, strides=stride_l4, padding='same') (out_cnn)
            out_cnn = Conv1D(filters=num_l4, kernel_size=kernel_l4, strides=stride_l4, padding='same') (out_cnn)
            out_cnn = Activation('relu') (out_cnn)
            out_cnn = BatchNormalization() (out_cnn)
            out_cnn = MaxPooling1D(pool_size=pool_size) (out_cnn)
            
                    


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            out_cnn = GlobalMaxPool1D() (out_cnn)
        elif globalpool_opt == 'ave':
            out_cnn = GlobalAveragePooling1D() (out_cnn)    
        out_cnn = Dropout(dropout_fc) (out_cnn)
        
        
        out = concatenate([out_fnn, out_cnn])
            
        if n_dense>=1:
            out = Dense(dense1, activation='tanh') (out)
            out = Dropout(dropout1) (out)
            
        if n_dense==2:
            out = Dense(dense2, activation='tanh') (out)
            out = Dropout(dropout2) (out)
            
        out = Dense(1, activation='sigmoid') (out)
        
        
        model = Model(inputs=[inp_cnn, inp_fnn], outputs=[out])


        # model 학습 설정
        try:
            model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_trains, y_train_bin, sample_weight=train_w_samp, validation_data=(x_vals, y_val_bin, val_w_samp), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])
        except:
            os.rmdir(odir)
            #os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            itrial -= 1
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue
            
            
    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_tests).flatten()

    
    # acc 계산
    acc = metrics.Accuracy()
    acc.update_state(y_pred>=0.5, y_test_bin, sample_weight=test_w_samp)
    acc_val = acc.result().numpy()
    test_acc.append(acc_val)
    
    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred, sample_weight=test_w_samp)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)

    # auprc 
    precision, recall, _ = precision_recall_curve(y_test_bin, y_pred, sample_weight=test_w_samp)
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)

    
    # rename
    os.rename(odir, rootdir+'/roc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_roc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_roc(max_idx), random_settings(max_idx)))


random search 0/400
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:C

Epoch 6/100
169/169 [==============================] - ETA: 0s - loss: 1.4514 - acc: 0.7895 - auc: 0.7026
Epoch 00006: val_loss improved from 1.49527 to 1.49081, saving model to ECG/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_120min/batch=1024,c1=256,c2=32,c3=32,filt1=5,filt2=5,filt3=3,str1=1,str2=1,str3=2,conv_double=True,globalpool=max,dropout=0.4,dnodes=16,dropout=0,dnodes=0,dropout=0/weights.hdf5
169/169 [==============================] - 96s 566ms/step - loss: 1.4514 - acc: 0.7895 - auc: 0.7026 - val_loss: 1.4908 - val_acc: 0.7761 - val_auc: 0.6418
Epoch 7/100
169/169 [==============================] - ETA: 0s - loss: 1.4458 - acc: 0.7896 - auc: 0.7115
Epoch 00007: val_loss did not improve from 1.49081
169/169 [==============================] - 95s 565ms/step - loss: 1.4458 - acc: 0.7896 - auc: 0.7115 - val_loss: 1.5042 - val_acc: 0.7727 - val_auc: 0.7208
Epoch 8/100
169/169 [==============================] - ETA: 0s - loss: 1.4454 - acc: 0.7888 - auc: 0.7177
Epoch 00008

169/169 [==============================] - ETA: 0s - loss: 1.4200 - acc: 0.7799 - auc: 0.7716- ETA: 0s - loss: 1.4227 - acc: 0.7795 - auc: 0.7
Epoch 00002: val_loss did not improve from 1.85637
169/169 [==============================] - 4s 26ms/step - loss: 1.4200 - acc: 0.7799 - auc: 0.7716 - val_loss: 2.7283 - val_acc: 0.4549 - val_auc: 0.6316
Epoch 3/100
167/169 [============================>.] - ETA: 0s - loss: 1.3381 - acc: 0.7902 - auc: 0.7996
Epoch 00003: val_loss did not improve from 1.85637
169/169 [==============================] - 4s 26ms/step - loss: 1.3381 - acc: 0.7902 - auc: 0.7996 - val_loss: 2.0013 - val_acc: 0.7378 - val_auc: 0.6331
Epoch 4/100
169/169 [==============================] - ETA: 0s - loss: 1.2788 - acc: 0.8007 - auc: 0.8216
Epoch 00004: val_loss did not improve from 1.85637
169/169 [==============================] - 4s 25ms/step - loss: 1.2788 - acc: 0.8007 - auc: 0.8216 - val_loss: 2.3602 - val_acc: 0.6090 - val_auc: 0.6763
random search 5/400
INFO:tenso

Epoch 1/100
169/169 [==============================] - ETA: 0s - loss: 1.5734 - acc: 0.7488 - auc: 0.7330
Epoch 00001: val_loss improved from inf to 3.87019, saving model to ECG/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_120min/batch=1024,c1=256,c2=256,c3=128,filt1=3,filt2=5,filt3=5,str1=3,str2=2,str3=2,conv_double=False,globalpool=ave,dropout=0.2,dnodes=16,dropout=0,dnodes=0,dropout=0/weights.hdf5
169/169 [==============================] - 28s 166ms/step - loss: 1.5734 - acc: 0.7488 - auc: 0.7330 - val_loss: 3.8702 - val_acc: 0.3431 - val_auc: 0.6158
Epoch 2/100
169/169 [==============================] - ETA: 0s - loss: 1.3257 - acc: 0.7949 - auc: 0.8051
Epoch 00002: val_loss did not improve from 3.87019
169/169 [==============================] - 22s 131ms/step - loss: 1.3257 - acc: 0.7949 - auc: 0.8051 - val_loss: 5.9671 - val_acc: 0.3431 - val_auc: 0.6099
Epoch 3/100
169/169 [==============================] - ETA: 0s - loss: 1.1952 - acc: 0.8150 - auc: 0.8515
Epoch 00003:

Epoch 11/100
169/169 [==============================] - ETA: 0s - loss: 1.2348 - acc: 0.8036 - auc: 0.8412
Epoch 00011: val_loss improved from 1.57193 to 1.56605, saving model to ECG/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_120min/batch=1024,c1=64,c2=32,c3=32,filt1=3,filt2=5,filt3=5,str1=1,str2=3,str3=1,conv_double=True,globalpool=max,dropout=0.2,dnodes=0,dropout=0,dnodes=0,dropout=0/weights.hdf5
169/169 [==============================] - 17s 102ms/step - loss: 1.2348 - acc: 0.8036 - auc: 0.8412 - val_loss: 1.5661 - val_acc: 0.7609 - val_auc: 0.7116
Epoch 12/100
169/169 [==============================] - ETA: 0s - loss: 1.2021 - acc: 0.8091 - auc: 0.8524
Epoch 00012: val_loss did not improve from 1.56605
169/169 [==============================] - 17s 101ms/step - loss: 1.2021 - acc: 0.8091 - auc: 0.8524 - val_loss: 1.7778 - val_acc: 0.7570 - val_auc: 0.6961
Epoch 13/100
169/169 [==============================] - ETA: 0s - loss: 1.1542 - acc: 0.8170 - auc: 0.8655
Epoch 0001

...

Epoch 2/100
168/169 [============================>.] - ETA: 0s - loss: 1.5442 - acc: 0.7634 - auc: 0.7272
Epoch 00002: val_loss improved from 2.01818 to 1.95746, saving model to ECG/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_120min/batch=1024,c1=128,c2=32,c3=0,filt1=3,filt2=3,filt3=0,str1=2,str2=3,str3=0,conv_double=True,globalpool=ave,dropout=0.4,dnodes=64,dropout=0.3,dnodes=0,dropout=0/weights.hdf5
169/169 [==============================] - 14s 83ms/step - loss: 1.5440 - acc: 0.7635 - auc: 0.7273 - val_loss: 1.9575 - val_acc: 0.7084 - val_auc: 0.6731
Epoch 3/100
168/169 [============================>.] - ETA: 0s - loss: 1.4546 - acc: 0.7821 - auc: 0.7400
Epoch 00003: val_loss improved from 1.95746 to 1.90662, saving model to ECG/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_120min/batch=1024,c1=128,c2=32,c3=0,filt1=3,filt2=3,filt3=0,str1=2,str2=3,str3=0,conv_double=True,globalpool=ave,dropout=0.4,dnodes=64,dropout=0.3,dnodes=0,dropout=0/weights.hdf5
169/169 [======

Epoch 7/100
169/169 [==============================] - ETA: 0s - loss: 1.5189 - acc: 0.7827 - auc: 0.7161
Epoch 00007: val_loss improved from 1.56388 to 1.53515, saving model to ECG/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_120min/batch=1024,c1=128,c2=32,c3=128,filt1=3,filt2=3,filt3=5,str1=1,str2=1,str3=1,conv_double=True,globalpool=max,dropout=0.1,dnodes=16,dropout=0.5,dnodes=0,dropout=0/weights.hdf5
169/169 [==============================] - 45s 264ms/step - loss: 1.5189 - acc: 0.7827 - auc: 0.7161 - val_loss: 1.5351 - val_acc: 0.7703 - val_auc: 0.6867
Epoch 8/100
169/169 [==============================] - ETA: 0s - loss: 1.5100 - acc: 0.7842 - auc: 0.7199
Epoch 00008: val_loss improved from 1.53515 to 1.51478, saving model to ECG/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_120min/batch=1024,c1=128,c2=32,c3=128,filt1=3,filt2=3,filt3=5,str1=1,str2=1,str3=1,conv_double=True,globalpool=max,dropout=0.1,dnodes=16,dropout=0.5,dnodes=0,dropout=0/weights.hdf5
169/169 [=

Epoch 5/100
336/338 [============================>.] - ETA: 0s - loss: 1.3728 - acc: 0.7887 - auc: 0.7799
Epoch 00005: val_loss improved from 1.64560 to 1.61089, saving model to ECG/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_120min/batch=512,c1=64,c2=32,c3=32,filt1=3,filt2=3,filt3=5,str1=3,str2=2,str3=3,conv_double=True,globalpool=max,dropout=0,dnodes=0,dropout=0,dnodes=0,dropout=0/weights.hdf5
338/338 [==============================] - 8s 24ms/step - loss: 1.3728 - acc: 0.7887 - auc: 0.7799 - val_loss: 1.6109 - val_acc: 0.7596 - val_auc: 0.6865
Epoch 6/100
337/338 [============================>.] - ETA: 0s - loss: 1.3514 - acc: 0.7905 - auc: 0.7889
Epoch 00006: val_loss did not improve from 1.61089
338/338 [==============================] - 8s 23ms/step - loss: 1.3513 - acc: 0.7905 - auc: 0.7889 - val_loss: 2.0570 - val_acc: 0.5714 - val_auc: 0.6921
Epoch 7/100
336/338 [============================>.] - ETA: 0s - loss: 1.3354 - acc: 0.7926 - auc: 0.7958
Epoch 00007: val_los

Epoch 7/100
169/169 [==============================] - ETA: 0s - loss: 1.4854 - acc: 0.7830 - auc: 0.6982
Epoch 00007: val_loss improved from 1.53861 to 1.52463, saving model to ECG/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_120min/batch=1024,c1=32,c2=256,c3=0,filt1=3,filt2=3,filt3=0,str1=2,str2=1,str3=0,conv_double=True,globalpool=max,dropout=0.4,dnodes=32,dropout=0,dnodes=32,dropout=0/weights.hdf5
169/169 [==============================] - 15s 91ms/step - loss: 1.4854 - acc: 0.7830 - auc: 0.6982 - val_loss: 1.5246 - val_acc: 0.7687 - val_auc: 0.6382
Epoch 8/100
169/169 [==============================] - ETA: 0s - loss: 1.4769 - acc: 0.7844 - auc: 0.7037
Epoch 00008: val_loss did not improve from 1.52463
169/169 [==============================] - 15s 90ms/step - loss: 1.4769 - acc: 0.7844 - auc: 0.7037 - val_loss: 1.5661 - val_acc: 0.7617 - val_auc: 0.6598
Epoch 9/100
169/169 [==============================] - ETA: 0s - loss: 1.4684 - acc: 0.7854 - auc: 0.7058
Epoch 00009: 

Epoch 7/100
169/169 [==============================] - ETA: 0s - loss: 1.0850 - acc: 0.8419 - auc: 0.8807
Epoch 00007: val_loss did not improve from 1.80537
169/169 [==============================] - 18s 109ms/step - loss: 1.0850 - acc: 0.8419 - auc: 0.8807 - val_loss: 1.9063 - val_acc: 0.7381 - val_auc: 0.6886
random search 90/400
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Epoch 1/100
338/338 [==============================] - ETA: 0s - loss: 1.6368 - acc: 0.7345 - auc: 0.7161
Epoch 00001: val_loss improved from inf to 1.83902, saving model to ECG/1D_CNN_3layers_sample-weighted_3class_lowess_filtered_120min/batch=512,c1=128,c2=128,c3=0,filt1=3,filt2=5,filt3=0,str1=2,str2=2,str3=0,conv_double=True,globalpool=ave,dropout=0.2,dnodes=0,dropout=0,dnodes=0,dropout=0/weights.hdf5
338/338 [==============================] - 23s 68ms/step - loss: 1.6368 - acc: 0.7345 - auc: 0.7161 - val_los

In [11]:
from numba import cuda
cuda.select_device(3)
cuda.close()

## 3 class

In [15]:
from keras import metrics
from keras.layers import LeakyReLU, ReLU
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# random search for hyperparameter
ntrial = 200
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_rmse, test_acc = [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    

    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2

        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={}, c1={}, c2={}, c3={}, c4={}, c1filts={}, c2filts={}, c3filts={}, c4filts={}, conv_double={}, globalpool_opt={}, dropout={}, dnodes={}, dropout={}'.format(batch_size, num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4, conv_double, globalpool_opt, dropout_cnn, dense_node, dropout_fc)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/model.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2", "/gpu:3"])
    with strategy.scope():
        # build a model
        model = Sequential()

        conv_act = True
        if conv_act:
            act = 'relu'
        else:
            act = None

        # c1 layer
        if conv_double:
            model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same'))
        model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))

        # c2 layer
        if conv_double:
            model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same'))
        model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c3 layer
        if conv_double:
            model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same'))
        model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c4 layer
        if conv_double:
            model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same'))
        model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            model.add(GlobalMaxPool1D())
        elif globalpool_opt == 'ave':
            model.add(GlobalAveragePooling1D())
            
            
        if dense_node != 0:
            model.add(Dropout(dropout_cnn))
            model.add(Dense(dense_node, activation='tanh'))
        model.add(Dropout(dropout_fc))
        model.add(Dense(3, activation='softmax'))

        try:
            # model 학습 설정
            model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_train, y_train_class, validation_data=(x_val, y_val_class), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])
        except:
            os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue

    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test)

    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve((y_test_class[:,1]+y_test_class[:,2])>=1, y_pred[:,1]+y_pred[:,2])
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)
    
    # auprc 
    precision, recall, _ = precision_recall_curve((y_test_class[:,1]+y_test_class[:,2])>=1, y_pred[:,1]+y_pred[:,2])
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)
    
    # acc 계산
    l_test = np.argmax(y_test_class, axis=1)
    l_pred = np.argmax(y_pred, axis=1)
    acc_val = accuracy_score(l_test, l_pred)
    test_acc.append(acc_val)
    
    # rename
    os.rename(odir, rootdir+'/auc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_auc(max_idx), random_settings(max_idx)))

random search 0/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
random search 1/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
random search 2/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
random search 3/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
random search 4/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
random search 5/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/d

random search 49/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
random search 50/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
random search 51/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
random search 52/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
random search 53/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
random search 54/200
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/ta

KeyboardInterrupt: 

In [16]:
from numba import cuda
cuda.select_device(2)
cuda.close()

## Regression

In [ ]:
from keras import metrics
# random search for hyperparameter
ntrial = 100
train_errs, val_errs = [], []
test_auc, test_rmse, test_acc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    

    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2

        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={}, c1={}, c2={}, c3={}, c4={}, c1filts={}, c2filts={}, c3filts={}, c4filts={}, conv_double={}, globalpool_opt={}, dropout={}, dnodes={}, dropout={}'.format(batch_size, num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4, conv_double, globalpool_opt, dropout_cnn, dense_node, dropout_fc)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/model.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    with strategy.scope():
        # build a model
        model = Sequential()

        conv_act = True
        if conv_act:
            act = 'relu'
        else:
            act = None

        # c1 layer
        if conv_double:
            model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same'))
        model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))

        # c2 layer
        if conv_double:
            model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same'))
        model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c3 layer
        if conv_double:
            model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same'))
        model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c4 layer
        if conv_double:
            model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same'))
        model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            model.add(GlobalMaxPool1D())
        elif globalpool_opt == 'ave':
            model.add(GlobalAveragePooling1D())
            
            
        if dense_node != 0:
            model.add(Dropout(dropout_cnn))
            model.add(Dense(dense_node, activation='sigmoid'))
        model.add(Dropout(dropout_fc))
        model.add(Dense(1, activation='sigmoid'))


        # model 학습 설정
        model.compile(loss='mse', optimizer=Adam(lr=learning_rate), metrics=["mean_absolute_error", tf.keras.metrics.AUC()])
        hist = model.fit(x_train, y_train/10, validation_split=0.1, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                            EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])


    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test).flatten()

    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_auc.append(roc_auc)
    # RMSE 계산
    model_err = metrics.RootMeanSquaredError()
    model_err.update_state(y_test, y_pred)
    rmse_val = model_err.result().numpy()
    test_rmse.append(rmse_val)
    # acc 계산
    acc_val = np.mean((y_pred*10>=5)==y_test_bin)
    test_acc.append(acc_val)
    # rename
    os.rename(odir, rootdir+'/auc{:.4f}_{}_rmse{:.4f}_acc{:.2f}'.format(roc_auc, odir_f, rmse_val, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_auc(max_idx), random_settings(max_idx)))

# Results

## 1D-CNN model 1
[input]-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(global maxpool)-dropout-(dense)-dropout-[output]
<br>or conv-conv-bn-maxpool

### Best Result

In [6]:
max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}\ninfo: {}'.format(test_auc[max_idx], random_settings[max_idx]))


Best Model roc:0.7853
info: batch=256, c1=64, c2=128, c3=32, c4=256, c1filts=11, c2filts=7, c3filts=13, c4filts=5, conv_double=False, globalpool_opt=ave, dropout=0.1, dnodes=0, dropout=0


### Top 10 Result

In [5]:
# top 10 model
topid= sorted(range(len(test_auc)),key= lambda i: test_auc[i])[-10:]

for i in range(10):
    print('Top {} Model: roc {:.4f}   train mse {:.4f}  val mse {:4f}'.format(i+1, np.array(test_auc)[topid[9-i]], np.array(train_errs)[topid[9-i]], np.array(val_errs)[topid[9-i]]))
    print(' {}\n'.format(np.array(random_settings[topid[9-i]])))

    #np.array(test_auc)[topid], np.array(random_settings)[topid]

Top 1 Model: roc 0.7853   train mse 0.2673  val mse 0.482166
 batch=256, c1=64, c2=128, c3=32, c4=256, c1filts=11, c2filts=7, c3filts=13, c4filts=5, conv_double=False, globalpool_opt=ave, dropout=0.1, dnodes=0, dropout=0

Top 2 Model: roc 0.7822   train mse 0.1443  val mse 0.441228
 batch=512, c1=32, c2=32, c3=512, c4=512, c1filts=3, c2filts=5, c3filts=5, c4filts=11, conv_double=False, globalpool_opt=ave, dropout=0.4, dnodes=8, dropout=0.1

Top 3 Model: roc 0.7808   train mse 0.4180  val mse 0.417034
 batch=256, c1=256, c2=512, c3=256, c4=32, c1filts=3, c2filts=3, c3filts=9, c4filts=15, conv_double=True, globalpool_opt=max, dropout=0.5, dnodes=16, dropout=0.2

Top 4 Model: roc 0.7781   train mse 0.4007  val mse 0.428640
 batch=1024, c1=32, c2=256, c3=128, c4=512, c1filts=15, c2filts=15, c3filts=13, c4filts=9, conv_double=True, globalpool_opt=max, dropout=0, dnodes=64, dropout=0.5

Top 5 Model: roc 0.7773   train mse 0.3908  val mse 0.438873
 batch=128, c1=32, c2=32, c3=512, c4=64, c1fi

## 1D-CNN model 2
[input]-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(global max or ave or flatten)-dropout-(dense)-dropout-[output]

In [52]:
max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}  rmse:{:.4f}  acc:{:.2f}\n: {}'.format(test_auc[max_idx], test_rmse[max_idx], test_acc[max_idx], random_settings[max_idx]))


Best Model roc:0.7496  rmse:4.2166  acc:0.66
: batch=256, c1nodes=64, c2nodes=128, c1filts=11, c2filts=8, globalpool_opt=ave, dropout=0.3, dnodes=8, dropout=0.5


In [53]:
# top 10 model
topid= sorted(range(len(test_auc)),key= lambda i: test_auc[i])[-10:]

for i in range(10):
    print('Top {} Model: roc {:.4f}  rmse:{:.4f}  acc:{:.2f},  train mse {:.4f}  val mse {:4f}'.format(i+1, np.array(test_auc)[topid[9-i]],np.array(test_rmse)[topid[9-i]], np.array(test_acc)[topid[9-i]], np.array(train_errs)[topid[9-i]], np.array(val_errs)[topid[9-i]]))
    print(' {}\n'.format(np.array(random_settings[topid[9-i]])))

    #np.array(test_auc)[topid], np.array(random_settings)[topid]

Top 1 Model: roc 0.7496  rmse:4.2166  acc:0.66,  train mse 0.0415  val mse 0.039510
 batch=256, c1nodes=64, c2nodes=128, c1filts=11, c2filts=8, globalpool_opt=ave, dropout=0.3, dnodes=8, dropout=0.5

Top 2 Model: roc 0.7492  rmse:4.2416  acc:0.59,  train mse 0.0394  val mse 0.040106
 batch=128, c1nodes=64, c2nodes=512, c1filts=3, c2filts=7, globalpool_opt=ave, dropout=0.3, dnodes=32, dropout=0.5

Top 3 Model: roc 0.7426  rmse:4.2331  acc:0.62,  train mse 0.0400  val mse 0.039092
 batch=256, c1nodes=64, c2nodes=64, c1filts=3, c2filts=6, globalpool_opt=max, dropout=0.1, dnodes=32, dropout=0.5

Top 4 Model: roc 0.7426  rmse:4.2217  acc:0.64,  train mse 0.0403  val mse 0.036776
 batch=1024, c1nodes=128, c2nodes=64, c1filts=12, c2filts=10, globalpool_opt=ave, dropout=0.3, dnodes=8, dropout=0.5

Top 5 Model: roc 0.7412  rmse:4.2421  acc:0.60,  train mse 0.0408  val mse 0.044279
 batch=128, c1nodes=64, c2nodes=512, c1filts=10, c2filts=12, globalpool_opt=ave, dropout=0.1, dnodes=16, dropout=0.

In [ ]:
def build_model(num_l1=256, kernel_l1 = 10, bool_flatten=False, dropout=0.2):
    #strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    #with strategy.scope():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    model = Sequential()
    #num_l2=64, num_l3=64, kernel_l2=3, kernel_l3=3
    num_l2=64
    num_l3=64
    kernel_l2=3
    kernel_l3=3

    #for (num_node, kernel_size) in conv_layers:
    #    model.add(Conv1D(filters=num_node, kernel_size=kernel_size, padding='valid'))
    #    model.add(BatchNormalization())
    #    model.add(MaxPooling1D(pool_size=2))

    # Conv Layer 1
    model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # Conv Layer 2
    model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # Conv Layer 3
    model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # FC layer 이전의 작업
    if bool_flatten:
        model.add(Flatten())
    else:
        model.add(GlobalMaxPool1D())

    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


return model